# Setting up MIRACL on Sherlock


## Download MIRACL to Sherlock

Log in to Sherlock:

```
$ ssh -Y username@sherlock.stanford.edu
```

Start an interactive session:

```
$ sdev
```

Move to your scratch folder:

```
$ cd SCRATCH
```

Pull (download) Singularity container:

```
$ singularity pull miracl_latest.sif library://aiconslab/miracl/miracl:latest
```

## Copy data to Sherlock

Copy a folder called, e.g. `input_clar` with tiff files that you want to register to the Allen Atlas using `scp`:

```
$ scp -r input_clar sherlock.stanford.edu:/scratch/users/<username>/clarity_registration/.
```

or `rsync`:

```
$ rsync -avPhz input_clar sherlock.stanford.edu:/scratch/users/<username>/clarity_registration/.
```

## Running MIRACL in an interactive session

To start an interactive session, shell into your Singularity container:

```
$ singularity shell miracl_latest.sif bash
```

Within the shell, load the GUI:

```
$ miraclGUI
```

Or use the command-line:

```
$ miracl lbls stats -h
```

## Running MIRACL commands with `singularity exec`

Singularity can also run commands without logging into it via a shell. To run commands that are available in the Singularity container without logging in, you can use `exec` with the following syntax:

```
singularity exec [exec options...] <container> <command>
```

Running a MIRACL function, e.g. registering an image to Allen Atlas, you can run:

```
singularity exec miracl_latest.sif clar_allen -i data/SHIELD_03x_down_autoflor_chan.nii.gz -o ALS -m combined -v 10
```

To open the MIRACL GUI run:

```
singularity exec miracl_latest.sif miraclGUI
```

## Using Apptainer instead of Singularity

As of 11/03/2024, Apptainer commands are still backwards compatible with Singularity commands which means that our Singluarity image of `MIRACL` can be used with Apptainer. To do so, just use the instructions provided for Singularity but instead of using the `singularity` command on the cli, substitute it with the `apptainer` command.

E.g., running a registration with our MIRACL Singularity container using Apptainer, you would use:

```
apptainer exec miracl_latest.sif clar_allen -i data/SHIELD_03x_down_autoflor_chan.nii.gz -o ALS -m combined -v 10
```

# Using MIRACL with Singularity on Compute Canada


## Copy data to Compute Canada



For example, to copy a folder called `input_clar` containing tiff files you want to register to the Allen Atlas use:

```
$ scp -r input_clar niagara.computecanada.edu:/scratch/<username>/.
```

or

```
$ rsync -avPhz input_clar <username>@niagara.computecanada.edu:/scratch/<username>/.
```


## Log in to a Compute Canada server

Log in to the Compute Canada server you copied your data to:

```
$ ssh -XY <username>@niagara.computecanada.edu
```


## Setting up and using MIRACL on Compute Canada

Load the specific Singularity module you would like to use (e.g. Singularity 3.5):

```
$ module load singularity/3.5
```

Since MIRACL will take up a significant amount of space, it is recommended to download and work with the MIRACL Singularity container in the scratch directory. First, navigate there:

```
$ cd $SCRATCH
```

Then pull (download) the Singularity container:

```
$ singularity pull miracl_latest.sif library://aiconslab/miracl/miracl:latest
```

> `singularity pull` requires Singularity version `3.0.0` or higher.

> If you have a particular Singularity container of MIRACL that you want to use on Compute Canada, just copy it to the servers directly using e.g. `scp` or `rsync` instead of pulling (downloading) the latest version of MIRACL from the Singularity registry

Start the MIRACL Singularity container with the default folders mounted:

```
$ singularity shell miracl_latest.sif bash
```

Singularity will automatically mount your scratch folder to your container. If you need to mount a specific directory into a specific location, use the following:

```
$ singularity shell -B <location_outside_container>/<source_mount>:<location_in_container>/<target_mount> miracl_latest.sif bash
```

Once you are logged in to the container, load the GUI from the shell:

```
$ miraclGUI
```

Or use MIRACL from the command line. For example, run MIRACL's CLARITY registration workflow on the folder that you copied over previously:

```
$ miracl flow reg_clar -f input_clar -n "-d 5 -ch autoflo" -r "-o ARS -m combined -v 25"
```

## `#SBATCH` scripting

For #SBATCH scripting on CC or Sherlock, you would create a shell script, e.g. simple_job.sh, that contains all instruction needed to automate your analysis.

The first part contains SBATCH commands that are used to interact with the SLURM cluster directly:



```
 #!/bin/bash
 #SBATCH --account=def-mgoubran
 #SBATCH --nodes=1
 #SBATCH --ntasks=1
 #SBATCH --cpus-per-task=16
 #SBATCH --mem=500GB
 #SBATCH --time=1-00:00
 #SBATCH --mail-user=cc_user@gmail.com
 #SBATCH --mail-type=BEGIN
 #SBATCH --mail-type=FAIL
 #SBATCH --mail-type=END
 #SBATCH --out=/home/cc_user/projects/rrg-mgoubran/experiment
```
These commands are used to set the parameters for the allocation you request on CC. For example, `mem` determines how much RAM your batch jobs will have available to them, in this case `500GB`.

The next part of the script contains the actual commands for the analysis. In this case we are using MIRACL to convert a stack of TIFF images into a nifti image using `miracl conv tiff_nii`:

```
module load singularity

# Change into directory you want to run the conversion function from
cd /home/cc_user/scratch/experiment/registration

# Run the conversion function with Singularity
singularity exec -B /project/6061906/dataset:/data miracl_latest.sif miracl  \
conv tiff_nii -f /data/Ex_561_Em_600_stitched \
-w /data/Ex_561_Em_600_stitched_downsample_03x_nii/ \
-o SHIELD -d 3 -ch signal -vx 3.5 -vz 4
```

Here the `-B` flag is used to mount the directory containing the data, i.e. the stack of TIFF images, into the Singularity container to make it accessible to the MIRACL command run inside of it. We are using the `miracl_latest.sif` container and specify the flags needed to run the conversion as part of the MIRACL command.

To run the script, i.e. request the location and once granted run the analysis, use:

```
$ sbatch simple_job.sh
Submitted batch job 123456
```